In [ ]:
# from google.colab import drive
# from google.colab import files
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install tensorflow

  Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
Using cached tensorflow-2.17.0-cp312-cp312-win_amd64.whl (2.0 kB)
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
  

ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\nhyir\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\nhyir\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\nhyir\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\nhyir\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nhyir\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 479, in read
    s = self.fp.read


   ---- ---------------------------------- 47.8/385.2 MB 266.0 kB/s eta 0:21:09
   ---- ---------------------------------- 47.8/385.2 MB 266.0 kB/s eta 0:21:09
   ---- ---------------------------------- 47.8/385.2 MB 266.0 kB/s eta 0:21:09
   ---- ---------------------------------- 47.8/385.2 MB 266.0 kB/s eta 0:21:09
   ---- ---------------------------------- 47.8/385.2 MB 266.0 kB/s eta 0:21:09
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- ---------------------------------- 47.8/385.2 MB 261.3 kB/s eta 0:21:31
   ---- -------------------------------

In [ ]:
from tensorflow.keras.applications import InceptionV3
import tensorflow as tf
import keras as keras
import pickle
import keras.backend as K
import cv2
import os
import shutil

## Splitting video into frames

In [ ]:
def get_frames(uploaded):
  if len(uploaded)==0:
    print("No file uploaded")
    return None
  else:
    filename = next(iter(uploaded))
    shutil.rmtree('frames')
    os.makedirs('frames',exist_ok=True)
    cap = cv2.VideoCapture(str(filename))
    frame_count = 0
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame_count += 1
      frame_filename = f'frames/frame_{frame_count}.jpg'
      cv2.imwrite(frame_filename, frame)
    cap.release()
    frame_filenames = sorted(os.listdir('frames'))
    print(f"Extracted {frame_count} frames.")
    return frame_filenames

In [ ]:
def detect_all_objects(frame_filenames):
  if frame_filenames==None:
    print("No frames found")
    return None, None
  else:
    frame_obj_dict = {}
    for frame_filename in frame_filenames:
        frame_path = os.path.join('frames', frame_filename)
        img = tf.io.read_file(frame_path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.cast(img, tf.float32)
        img = tf.image.resize(img, (299, 299))
        img = tf.keras.applications.inception_v3.preprocess_input(img)
        img_expanded = tf.expand_dims(img, axis=0)
        prediction = model.predict(img_expanded,verbose=0)
        decoded_predictions = tf.keras.applications.inception_v3.decode_predictions(prediction, top=3)
        img_dict = {}
        for i, (imagenet_id, label, score) in enumerate(decoded_predictions[0]):
              img_dict[i+1] = label
        frame_obj_dict[frame_filename] = img_dict

    all_objects = []
    for frame_name in frame_obj_dict.keys():
      for obj_id in frame_obj_dict[frame_name].keys():
        all_objects.append(frame_obj_dict[frame_name][obj_id])
    all_obj = list(set(all_objects))

    return frame_obj_dict, all_obj

In [ ]:
def search_object(search_query,frame_obj_dict,all_obj):
  if frame_obj_dict==None:
    print("No frames found")
    return None
  else:
    obj_frames = []
    for frame_name in frame_obj_dict.keys():
      if search_query in frame_obj_dict[frame_name].values():
        obj_frames.append(frame_name)
    import matplotlib.pyplot as plt
    if len(obj_frames)==0:
      print("Object doesn't exist!!!")
      print("\nChoose from the list below")
      for obj in all_obj:
        print(f'\n {obj}')
    else:
      for framee in obj_frames:
        frame_path = os.path.join('frames', framee)
        frame = cv2.imread(frame_path)
        print(f'\n{search_query} in {framee}')
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

In [ ]:
def obj_in_video(search_query):
  model = InceptionV3(weights='imagenet', include_top=True)
  uploaded = files.upload()
  filenames = get_frames(uploaded)
  fram_obj_dict, objs = detect_all_objects(filenames)
  search_object(search_query,fram_obj_dict,objs)

In [ ]:
import pickle
from tensorflow.keras.applications import InceptionV3

model = InceptionV3(weights='imagenet', include_top=True)
with open('inception_v3_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
obj_in_video("rubber_eraser")

96112376/96112376 [==============================] - 4s 0us/step


Saving passport_camera.mov to passport_camera.mov


FileNotFoundError: [Errno 2] No such file or directory: 'frames'